In [5]:
!nvidia-smi

Fri Jan  8 12:47:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-SXM2...  Off  | 00000000:05:00.0 Off |                    0 |
| N/A   51C    P0   178W / 300W |  10768MiB / 16280MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-SXM2...  Off  | 00000000:06:00.0 Off |                    0 |
| N/A   50C    P0    56W / 300W |   4547MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [17]:
from utils.dataset_loader import BasicDataset
# from eval import eval_net
from model.model import LogoDetection

import torch
from torch.utils.data import DataLoader
import os
from PIL import Image
import numpy as np


def get_model(load: str = None, model_path: str = None, checkpoint_dir: str = None):
    if load:
        if os.path.isfile(load):
            return load
    else:
        if model_path and os.path.isfile(model_path):
            return model_path
        else:
            if checkpoint_dir and os.path.isdir(checkpoint_dir):
                only_files = [f for f in os.listdir(checkpoint_dir) if
                              os.path.isfile(os.path.join(checkpoint_dir, f)) and '.pt' in f]
                return f'{checkpoint_dir}{os.path.sep}{only_files[0]}'


def predict_main(masks: list, target_imgs: list = None, bboxes_query: list = None, query_images: list = None,
                 custom_model_path: str = None, model_path: str = None, checkpint_dir: str = None):
    if target_imgs or bboxes_query or query_images:
        assert len(target_imgs) == len(bboxes_query) == len(query_images), \
            "Target images, bboxes and query images must have the same amount of elements"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LogoDetection(batch_norm=True, vgg_cfg='B')
    model.load_state_dict(
        torch.load(get_model(load=custom_model_path, model_path=model_path, checkpoint_dir=checkpint_dir),
                   map_location=device),
        strict=False
    )
    model.to(device=device)
    for img_idx, img in enumerate(target_imgs):
        dataset = BasicDataset.preprocess(target_img_path=img, query_full_img_path=query_images[img_idx],
                                          bbox_path=bboxes_query[img_idx], mask_img_path=masks[img_idx], skip_bbox_lines=1)
        # queries, targets,  = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=1, pin_memory=True)
        # for batch in dataset_loader:
        queries, targets = dataset['query'], dataset['target']
        queries = queries.to(device=device, dtype=torch.float32)
        targets = targets.to(device=device, dtype=torch.float32)
#        queries = np.expand_dims(queries, axis=0)
#        targets = np.expand_dims(targets, axis=0)
        queries = queries.unsqueeze(0)
        targets = targets.unsqueeze(0)
        print(f"queries.shape: {queries.shape}")
        print(f"targets.shape: {targets.shape}")
        with torch.no_grad():
            pred = model(queries, targets)
            pred_mask = pred.cpu().numpy()
            pred_mask_pil = Image.fromarray(pred_mask)
            true_mask_pil = Image.open(masks[img_idx])
            dst = Image.new('RGB', (true_mask_pil.width + pred_mask_pil.width, true_mask_pil.height))
            dst.paste(true_mask_pil, (0, 0))
            dst.paste(pred_mask_pil, (true_mask_pil.width, 0))
            dst.save(f"{img_idx}.jpg")

In [18]:
predict_main(
    masks=["/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/masks/adidas0/adidas1.jpg.bboxes.txt",
           "/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/masks/adidas0/adidas2.jpg.bboxes.txt"],
    target_imgs=["/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/jpg/adidas0/adidas1.jpg",
                 "/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/jpg/adidas0/adidas2.jpg"],
    bboxes_query=["/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/masks/adidas0/adidas1.jpg.bboxes.txt",
           "/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/masks/adidas0/adidas2.jpg.bboxes.txt"],
    query_images=["/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/jpg/adidas0/adidas1.jpg",
                 "/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/dataset/qmul_toplogo10/jpg/adidas0/adidas2.jpg"],
    custom_model_path="/home/nvidia/workspace/alessandroaurora/one-shot_logo_detection/data/stored_models/backups/CP_epoch14_lr_4e-5.pt"
)

RuntimeError: Error(s) in loading state_dict for LogoDetection:
	size mismatch for latent_repr.vgg16.7.weight: copying a param with shape torch.Size([64, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
	size mismatch for latent_repr.vgg16.10.weight: copying a param with shape torch.Size([64, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([128, 64, 3, 3]).
	size mismatch for latent_repr.vgg16.10.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for latent_repr.vgg16.19.weight: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([512, 512, 2, 2]).
	size mismatch for latent_repr.vgg16.19.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([512]).

In [ ]:
from utils.dataset_loader import BasicDataset
BasicDataset.preprocess(
    target_img_path=""
)